In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc1


In [4]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device,
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    val_labels = sampler.test_data[sampler.test_mask] 

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None 
    
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device,
            )
        if true_data is not None:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)], ignore_index=True
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)], ignore_index=True
            )

  0%|          | 0/331 [00:00<?, ?it/s]

epoch:   0 loss:0.667145 auc:0.4766
epoch:  20 loss:0.325609 auc:0.6612
epoch:  40 loss:0.244149 auc:0.6891
epoch:  60 loss:0.217981 auc:0.7237
epoch:  80 loss:0.208568 auc:0.7380
epoch: 100 loss:0.202662 auc:0.7425
epoch: 120 loss:0.196682 auc:0.7430
epoch: 140 loss:0.193314 auc:0.7407
epoch: 160 loss:0.191791 auc:0.7439
epoch: 180 loss:0.190721 auc:0.7418
epoch: 200 loss:0.190280 auc:0.7390
epoch: 220 loss:0.189033 auc:0.7382


  0%|          | 1/331 [00:03<18:20,  3.33s/it]

epoch: 240 loss:0.188883 auc:0.7363
Fit finished.
epoch:   0 loss:1.256336 auc:0.5346
epoch:  20 loss:0.321286 auc:0.4211
epoch:  40 loss:0.247298 auc:0.3380
epoch:  60 loss:0.219892 auc:0.6759
epoch:  80 loss:0.210107 auc:0.7175
epoch: 100 loss:0.203276 auc:0.7590
epoch: 120 loss:0.196638 auc:0.8255
epoch: 140 loss:0.194014 auc:0.8172
epoch: 160 loss:0.192638 auc:0.8199
epoch: 180 loss:0.192008 auc:0.8227
epoch: 200 loss:0.190858 auc:0.8172
epoch: 220 loss:0.191625 auc:0.8144
epoch: 240 loss:0.189388 auc:0.8089
epoch: 260 loss:0.190278 auc:0.8061


  1%|          | 2/331 [00:07<21:01,  3.83s/it]

...

epoch: 320 loss:0.181926 auc:0.9040
Fit finished.
epoch:   0 loss:1.138109 auc:0.4915
epoch:  20 loss:0.325752 auc:0.5177
epoch:  40 loss:0.247049 auc:0.4376
epoch:  60 loss:0.217616 auc:0.6632
epoch:  80 loss:0.207541 auc:0.6786
epoch: 100 loss:0.201300 auc:0.6979
epoch: 120 loss:0.195892 auc:0.7125
epoch: 140 loss:0.193015 auc:0.7199
epoch: 160 loss:0.191544 auc:0.7246
epoch: 180 loss:0.190622 auc:0.7261
epoch: 200 loss:0.190105 auc:0.7264
epoch: 220 loss:0.189249 auc:0.7271
epoch: 240 loss:0.188703 auc:0.7226
epoch: 260 loss:0.188392 auc:0.7208


 99%|█████████▉| 327/331 [13:02<00:13,  3.29s/it]

epoch: 280 loss:0.187888 auc:0.7109
Fit finished.
epoch:   0 loss:0.637271 auc:0.4053
epoch:  20 loss:0.272638 auc:0.5788
epoch:  40 loss:0.221000 auc:0.6513
epoch:  60 loss:0.210268 auc:0.6655
epoch:  80 loss:0.204222 auc:0.6938
epoch: 100 loss:0.197593 auc:0.7522
epoch: 120 loss:0.193824 auc:0.7796
epoch: 140 loss:0.192450 auc:0.7531
epoch: 160 loss:0.190975 auc:0.7416


100%|██████████| 331/331 [13:05<00:00,  2.37s/it]

epoch: 180 loss:0.194484 auc:0.5726
Fit finished.


In [8]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")